# DeepPavlov basics 
In this tutorial we will construct elementary components needed for working with different NLP tasks. We will go through typical data preprocessing pipeline which will be used in the next tutorials. This part is mostly about low-level elements of the library. In the end will construct a simple bot based on pattern matching and the library abstactions.

### Tutorial plan

1. [Install the library](#Install-the-library):
    - [on Linux](#Install-the-library)
    - [on Windows](#Install-the-library-on-Windows-using-Docker)
2. [Hello bot](#Hello-bot)
3. [Data](#Data):
    - [Parsing text data](#Parsing-text-data-into-a-machine-readable-dataset)
    - [Preparation of a dictionary](#Prepare-dictionaries)
    - [Dataset iterator](#Dataset-Iterator)

### Libraries

For this task you will need the following libraries:
 - [Tensorflow](https://www.tensorflow.org) — an open-source software library for Machine Intelligence.
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 - [DeepPavlov](https://github.com/deepmipt/deeppavlov) - open source library for Natural Language Processing
 


# Install the library

Currently only Linux platform and Python 3.6 is supported

- Create a virtual environment with Python 3.6

    `virtualenv -p python3.6 env`

- Activate the environment.

    `source ./env/bin/activate`

- Clone the repo and cd to project root

    `git clone https://github.com/deepmipt/DeepPavlov.git`
    
    `cd DeepPavlov`

- Install the requirements:

    `python setup.py develop`

- Install spacy dependencies:

    `python -m spacy download en`



# Install the library on Windows using Docker

First, install the Docker following these instructions:

https://docs.docker.com/docker-for-windows/install

Then go to console and get the container with the following command:

`docker pull altinsky/convai:deeppavlov`

Run the container:

`docker run -p 8888:8888 altinsky/convai:deeppavlov`

Navigate to http://127.0.0.1:8888/ in your browser.

To STOP the container run:

`docker stop`

# Hello bot

In this part we will construct a simple bot that relies on pattern matching to perform a conversation.

In [469]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
from deeppavlov.core.agent import Agent, HighestConfidenceSelector

A pattern matching skill is the simplest example of Natural Language Understanding component. It will search defined patterns through the text. Let's define some simple patterns:

In [470]:
hello = PatternMatchingSkill(['Hello world!'], patterns=["hi", "hello", "good day"])
bye = PatternMatchingSkill(['Goodbye world!', 'See you around'],
                           patterns=["bye", "chao", "see you"])
fallback = PatternMatchingSkill(["I don't understand, sorry", 'I can say "Hello world!"'])

If you provide some patterns to the PatternMatchingSkill it will return confidence = 1 when the skill finds the pattern in given text. If no patterns is provided then confidence 0.5 will be returned in any case.

The skills are used in the `Agent` which can be treated as a Dialog Manager. The agent must be provided with skills and the selector of skills. A simple skill selector is the HighestConfidenceSelector which will pick the skill with highest confidence.

In [471]:
HelloBot = Agent([hello, bye, fallback], skills_selector=HighestConfidenceSelector())

Since all processing in the library is performed on batches, we can pass a batch of requests to the bot. Let's try it out:

In [472]:
HelloBot(['Hello', 'Bye', 'Or not'])

['Hello world!', 'Goodbye world!', "I don't understand, sorry"]

**Exercise** 
- create a WhatIsYourName skill
- create new agent with this skill
- check that all works fine

In [473]:
name = 1# YOUR_CODE_HERE


whats_your_name = PatternMatchingSkill(['Whats your name?', 'I need information about you'],
                           patterns=["name", "information"])
#fallback = PatternMatchingSkill(["I don't understand, sorry", 'I can say "Hello world!"'])

In [474]:
NameBot = Agent([whats_your_name], skills_selector=HighestConfidenceSelector())
NameBot(['information'])

['I need information about you']

## Data
Deeppavlov library has functionality to download and decompress the data. For this purpose the `download_decompress` from `data.utils` is used. 
The following cell will download the CoNLL-2003 data for the Named Entity Recognition (NER) task and put it to the folder `data/`.

In [475]:
import deeppavlov
from deeppavlov.core.data.utils import download_decompress
download_decompress('http://lnsigo.mipt.ru/export/deeppavlov_data/conll2003_v2.tar.gz', 'data/')

2018-07-03 19:01:04.254 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): lnsigo.mipt.ru
2018-07-03 19:01:04.287 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://lnsigo.mipt.ru:80 "GET /export/deeppavlov_data/conll2003_v2.tar.gz HTTP/1.1" 511 None
2018-07-03 19:01:04.288 INFO in 'deeppavlov.core.data.utils'['utils'] at line 65: Downloading from http://lnsigo.mipt.ru/export/deeppavlov_data/conll2003_v2.tar.gz to /home/temkahap/Рабочий стол/CISS/DeepPavlov/data/conll2003_v2.tar.gz
506B [00:00, 1.48MB/s]
2018-07-03 19:01:04.291 INFO in 'deeppavlov.core.data.utils'['utils'] at line 149: Extracting data/conll2003_v2.tar.gz archive into data


ReadError: file could not be opened successfully

### Parsing text data into a machine-readable dataset 

We will work with a corpus which contains tweets with NE tags. A typical file with NER data contains lines with pairs of tokens (word or punctuation symbol) and tags separated by a whitespace. In many cases additional information such as POS-tags is included. 

Different documents are separated by lines **started** with **-DOCSTART-** token. Different sentences are separated by an empty line. Example:

    -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O

    Peter NNP B-NP B-PER
    Blackburn NNP I-NP I-PER

In this tutorial we will focus only on tokens and tags (first and last elements of the line) and drop POS information located between them.

We start by building a class *NerDatasetReader*  that provides functionality for reading the dataset. It returns a dictionary with fields *train*, *test*, and *valid*. Each field stores a list of samples. Each sample is a tuple of tokens and tags. Both tokens and tags are lists. The following example depicts the structure that should be returned by *read* method:

    {'train': [(['Mr.', 'Dwag', 'are', 'derping', 'around'], ['B-PER', 'I-PER', 'O', 'O', 'O']), ....],
     'valid': [...],
     'test': [...]}

There are three separate parts in the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.
 

Each of these parts is stored in a separate txt file.


In [476]:
from pathlib import Path

class NerDatasetReader:
    def read(self, data_path):
        data_parts = ['train', 'valid', 'test']
        extension = '.txt'
        dataset = {}
        for data_part in data_parts:
            file_path = Path(data_path) / Path(data_part + extension)
            dataset[data_part] = self.read_file(str(file_path))
        return dataset
            
    @staticmethod
    def read_file(file_path):
        
        # Use utf-8 encoding when open the file
        ######################################
        ########## YOUR CODE HERE ############
        ######################################
        with open(file_path) as f:
            final_list = []
            list_sent = []
            list_tags = []
            for f in f.readlines():
                t = f.split(' ')
                if t[0] == '-DOCSTART-':
                    list_sent = []
                    list_tags = []
                    continue
                if ((t[0] is not '\n') and (t[-1] is not '\n')):
                    list_sent.append(t[0])
                    list_tags.append(t[-1])#.strip('\n')[0])
                    #print (t[-1])
                if t[0] == '\n' and len(list_sent) > 0:
                    final_list.append((list_sent, list_tags))
                    list_sent = []
                    list_tags = []
        return final_list

In [477]:
dataset_reader = NerDatasetReader()

In [478]:
dataset = dataset_reader.read('data/')
assert len(dataset) == 3, 'The dataset must be a dict with three fields: train, test, and valid'
assert len(set(dataset) & {'train', 'test', 'valid'}) == 3, 'The dataset keys must be exactly train, test, and valid'
assert isinstance(dataset['train'][0][0][0], str) and isinstance(dataset['train'][0][0][1], str), 'Both tokens and tags must be strings'
assert len(dataset['train']) == 14041, 'there must be exactly 14041 samples in train'
assert len(dataset['valid']) == 3250, 'there must be exactly 3250 samples in train'
assert len(dataset['test']) == 3453, 'there must be exactly 3453 samples in test'

You should always understand what kind of data you deal with. For this purpose, you can print the data by running the code in the following cell:

In [479]:
for sample in dataset['train'][:2]:
    for token, tag in zip(*sample):
        print('%s\t%s' % (token, tag))
    print()

EU	B-ORG

rejects	O

German	B-MISC

call	O

to	O

boycott	O

British	B-MISC

lamb	O

.	O


Peter	B-PER

Blackburn	I-PER




You can find an implementation of the dataset reader that implemets the same interfaces in the library: [Conll2003DatasetReader](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/dataset_readers/conll2003_reader.py). The functionality of the presented code is wider and the `register` wrapper allows to use this component as a part of config file (will be discussed later).

### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Token indices will be used to address a row in embeddings matrix. The mapping for tags will be used to create one-hot ground-truth probability distribution vectors to compute the loss at the output of the network.

Now you need to implement the *Vocab* class which will return {token or tag}$\to${index} and vice versa. 

In [480]:
from collections import defaultdict, Counter
from itertools import chain
import numpy as np

In [481]:
class Vocab:
    def __init__(self,
                 special_tokens=tuple()):
        self.special_tokens = special_tokens
        self._t2i = defaultdict(lambda: 1)
        self._i2t = []
        
    def fit(self, tokens):
        count = 0
        self.freqs = Counter(chain(*tokens))
        # The first special token will be the default token
        for special_token in self.special_tokens:
            self._t2i[special_token] = count
            self._i2t.append(special_token)
            count += 1
        for token, freq in self.freqs.most_common():
            if token not in self._t2i:
                self._t2i[token] = count
                self._i2t.append(token)
                count += 1

    def __call__(self, batch, **kwargs):
        # Implement the vocab() method. The input could be a batch of tokens
        # or a batch of indices. A batch is a list of utterances where each
        # utterance is a list of tokens
        #pass
        ######################################
        ########## YOUR CODE HERE ############
        ######################################
        final_list = []
        for ind in batch:
            temp_list = []
            for item in ind:
                temp_list.append(self.__getitem__(item))
            final_list.append(temp_list)
        return final_list
        
        
    def __getitem__(self, key):
        # Implement the vocab[] method. The input could be a token
        # (string) or an index. You have to detect what type of data
        # is key and return. 
        ######################################
        ########## YOUR CODE HERE ############
        ######################################
        #if type(key) == 'str':
        if isinstance(key, str):
            return self._t2i[key]
        else:
            return self._i2t[key]
            
    
    def __len__(self):
        return len(self._i2t)


After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens
 - `'O'` for the tag vocab to place out of label tag to the first place with index 0

In [482]:
special_tokens = ['<UNK>']
special_tags = ['O\n']

token_vocab = Vocab(special_tokens)
tag_vocab = Vocab(special_tags)

Now we will fit the vocabularies on the *train* part of the data.

In [483]:
all_tokens_by_sentenses = [tokens for tokens, tags in dataset['train']]
all_tags_by_sentenses = [tags for tokens, tags in dataset['train']]

token_vocab.fit(all_tokens_by_sentenses)
tag_vocab.fit(all_tags_by_sentenses)
print (len(tag_vocab))
print (tag_vocab._t2i)

assert len(token_vocab) == 23624, 'There must be exactly 23624 in the token vocab!'
assert len(tag_vocab) == 9, 'There must be exactly 9 in the tag vocab!'

9
defaultdict(<function Vocab.__init__.<locals>.<lambda> at 0x7f62708fcae8>, {'O\n': 0, 'B-LOC\n': 1, 'B-PER\n': 2, 'B-ORG\n': 3, 'I-PER\n': 4, 'I-ORG\n': 5, 'B-MISC\n': 6, 'I-LOC\n': 7, 'I-MISC\n': 8})


In [484]:
r = []
for i in all_tags_by_sentenses:
    #print (np.unique(i))
    r+= list(np.unique(i))
print (np.unique(r))

['B-LOC\n' 'B-MISC\n' 'B-ORG\n' 'B-PER\n' 'I-LOC\n' 'I-MISC\n' 'I-ORG\n'
 'I-PER\n' 'O\n']


Try to get the indices. Keep in mind that we are working with batches of the following structure:
    
    [['utt0_tok0', 'utt1_tok1', ...], ['utt1_tok0', 'utt1_tok1', ...], ...]

In [485]:
indices_batch

[[10167, 6, 1, 7, 1, 1865]]

In [486]:
indices_batch = token_vocab([['How', 'to', 'cook', 'a', 'turnip', '?']])
assert len(indices_batch) == 1, 'the batch length must be 1'
assert isinstance(indices_batch[0][0], int), 'The batch must contain lists of ints!'

print(indices_batch)

[[10167, 6, 1, 7, 1, 1865]]


In [487]:
tag_indices_batch = tag_vocab([['O', 'O', 'O'], ['B-PER']])

assert len(tag_indices_batch) == 2, 'the batch length must be 2'
assert isinstance(tag_indices_batch[0][0], int), 'The batch must contain lists of ints!'

print(tag_indices_batch)

[[1, 1, 1], [1]]


Now we will try converting from indices to tokens.

In [488]:
token_vocab([np.random.randint(0, 512, size=10)])

[['authorities',
  'be',
  'new',
  'just',
  '10',
  'war',
  'United',
  'that',
  '7-5',
  'one']]

A similar vocabulary is already implemented in the [library](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/core/data/simple_vocab.py). It has extended functionality:
- token cutoff by frequency
- limitation of the vocabulary size
- saving and loading
- dict like dunders (\_\_contain\_\_, \_\_len\_\_, etc.)

### Dataset Iterator

Neural Networks are usually trained with batches. It means that weight updates of the network are based on several sequences at every single time. You have to iterate over the dataset and generate `x` and `y` batch by batch. The batch of `x`-s is a list of sentences of tokens like

    [['Yan', 'is', 'a', 'good', 'fellow],
     ['For', 'instance']]

and the tag sequence should be:

    [['B-PER', 'O', 'O', 'O', 'O'],
     ['O', 'O']]

An important concept in the batch generation is shuffling. Shuffling is taking sample from the dataset at random order. It is important to train on the shuffled data because large number consequetive samples of the same class may result in pure quality of the model.
    
The idea behind the iterator is to perform computation in the lazy way. Use yield generator expression to do so. An example of using yield for generator creation is provided below:

In [489]:
data = []

def iterator():
    data = [1, 2, 3]
    for d in data:
        yield d
            
print(iterator)
    
for i in iterator():
    print(i)

<function iterator at 0x7f6235a3c0d0>
1
2
3


Now create the `DatasetIterator`:

In [490]:
class DatasetIterator:
    def __init__(self, data):
        self.data = {
            'train': data['train'],
            'valid': data['valid'],
            'test': data['test']
        }

    def gen_batches(self, batch_size, data_type='train', shuffle=True):
        ######################################
        ########## YOUR CODE HERE ############
        ######################################
        #pass
        
        data_perm = list(range(len(dataset[data_type])))
        
        # Перемешанное
        if shuffle == True:
            random.shuffle(data_perm)
        
        x, y = [], []
        for i in range(batch_size):
            x.append(dataset[data_type][data_perm[i]][0]) 
            y.append(dataset[data_type][data_perm[i]][1])

        yield x,y
        
            


Create the dataset iterator from the loaded dataset

In [491]:
data_iterator = DatasetIterator(dataset)

Try it out:

In [492]:
x, y = next(data_iterator.gen_batches(2))

assert len(x) == 2, 'There must be two examples in the batch!'
assert len(y) == 2, 'There must be two examples in the batch!'
assert len(x[0]) == len(y[0]), 'The numbers of tokens and tags are different!'
assert isinstance(x[0][0], str), 'Token must be a string!'

This is a typical part of the data preprocessing pipeline. This parts will be used in the following tutorials. 